# 4. Advanced NumPy Modelling Examples

This notebook will involve no direct teaching - instead we are going to attempt to solve a number of difficult problems. These problems will attempt to range over a number of inter-disciplinary fields. Don't worry if you are not able to complete them all within the time of the workshop - they are meant to stretch your abilities, gain some useful NumPy experience and grow some inter-disciplinary knowledge. For these tasks we require that you **only** use NumPy arrays as this is considerably faster and the only tractable method in later examples.



In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Tasks

## Monte Carlo Integration

*Monte Carlo methods* rely primarily on repeated sampling to obtain numerical results, and are used extensively in any application that involves a probabilistic interpretation. They are particularly powerful in *high-dimensional* problems where deterministic methods for high dimensions become intractable. They are often coupled with other techniques such as **Markov Chains** once the probability distribution of a variable is parameterized.

Let's say we have a mathematical function that is difficult to integrate:

$$
f(x)=\sin^2 \left(\frac{1}{x(2-x)}\right)
$$

to see this, do a quick plot.

We see that the infinite oscillation as $f(x)$ draws towards 0 and 2 makes this function incredibly difficult to integrate using standard numerical methods (composite trapezoidal, simpsons).

However we can make use of the fact that this function is *bounded* at [0, 2], and **scatter** a large uniform random distribution $\cal N[0, 1]$ across this box. The fraction of them falling below the curve is approximately the integral we want to compute; hence:

$$
I=\int_a^b f(x) \ dx \quad \implies \quad I \simeq \frac{k A}{N}
$$

where $N$ is the total number of points considered, $k$ is the number falling below the curve, and $A$ is the area of the box. We know that $x \in [0, 2]$, and we choose $y \in [0, 1]$, giving $A=(y_1-y_0)(x_1-x_0)$.

### Task 1.

Write a function `monte_carlo_integrate()` which receives the function $f(x)$, the domain of $x$, the domain of $y$, and $N$. It should compute the integral of `f(x)` using the Monte Carlo method. Remember that the random numbers generated *must* be scaled into the domains of $x$ and $y$.

In [3]:
# write codes here

### Task 2.

Call `monte_carlo_integrate()` with the function described above, with $x \in [0, 2]$, $y \in [0, 1]$, and $N=10^5$. Is it reasonable?

In [4]:
# write codes here


### Task 3.

The area of a circle of radius 2 is $4\pi$. To help check the accuracy, let's calculate $\pi$ by calculating the area of a *quarter-circle* in $x,y \in [0, 2]$:

$$
\pi = \int_0^2 \sqrt{4-x^2} \ dx
$$

Repeat calls to `monte_carlo_integrate()` and generate integrals of Monte-Carlo integration with $N$. We recommend using $N=100 \times 2^i$ for $i=0,\dots,15$.

In [5]:
# write codes here

### Task 4.

Generate the loglog plot of $N$ against $E$, also known as the **convergence** of the algorithm, where:

$$
E= \lvert \pi-\hat \pi \rvert
$$

where $\hat \pi = I$ integral from our Monte-Carlo method.

In [6]:
# write codes here

### Task 5.

The coefficients for the slope (m) and intercept (b) can be calculated with `np.polyfit(N,E,1)`, remember to pass the $\log N$ and $\log E$! Plot the slope line through the points using loglog, using $y=mx+b$ for the $y$ variables. However given that the plot is in logspace, we calculate $y$ as:

$$
y=\exp(b)N^m
$$

Calculate and plot the slope of the plot.

In [7]:
# write codes here

## Linear Regression

For many real-world applications, given a set of data $X$ which we consider inputs or measurements observed, and $y$ which is considered an *continuous* output, or useful metric to inter, is there a set of latent coefficients/weights $w$ which when scaled to $X$ can infer $y$? This is the fundamental principle of **linear regression**. In this notation, capital letters $X$ describe *matrices*, and lower-case letters *w,y* describe vectors, with greek letters describing scalar coefficients. Linear models are henceforth:

$$
\bf y=w^Tx+\eta+\epsilon
$$

where $w$ is our slope/gradient, $x$ is the input, $\eta$ is the intercept and $\epsilon$ is the error.

We will however work in matrix notation, and to somplify the math we merge the intercept $\eta$ into the weights $w$, and add a bias column to $X$:

$$
\mathbf{X}=\left[\begin{matrix}
   x_{11} & x_{12} & \dots & x_{1m} & 1\\ 
            x_{21} & x_{22} & \dots & x_{2m} & 1\\ 
            \vdots & \vdots & \ddots & \vdots & \vdots \\ 
            x_{n1} & x_{n2} & \dots & x_{nm} & 1\\ 
  \end{matrix} \right], \qquad
  \mathbf{w}=\left[\begin{matrix}
    w_1 \\ w_2 \\ \vdots \\ w_n \\ \eta
  \end{matrix}\right]
$$

where $\bf x_1, \dots, x_n$ is a vector, $w_1, \dots, w_n$ are scalars. Therefore our new model is $\mathbf{y=Xw}$, with $p+1$ unknowns. In order to find the best $\bf w$, we minimize the difference between the values generated from $\bf Xw$ and $\bf y$, as:

$$
\mathbf{e} = \min \ \lvert \lvert \mathbf{Xw-y} \rvert \rvert^2
$$

to minimize this, we calculate the derivative with respect to each of the weights $\bf w$ (including intercept):

$$
\Delta_w \mathbf{e} = 2\mathbf{X^T}(\mathbf{Xw-y})
$$

Equating this to 0, and after some equation manipulation we get:

$$
\mathbf{X^TXw}=\mathbf{X^Ty} \\
\mathbf{w}=(\mathbf{X^TX})^{-1}\mathbf{X^Ty}
$$

### Task 1.

Write a function `least_squares()`, that given an input matrix $\mathbf{X}_{N,P}$ and output vector $\mathbf{y}_N$, *directly* calculates and returns the best weight coefficients $\mathbf{w}_P$. You can use `np.linalg.inv()` to calculate the matrix inverse. Remember to include the bias column to the $X$ matrix. To generate $X$ and $y$, use the function `generate_regression()` provided, you may choose to change some of the optional parameters.

In [5]:
def make_regression(n_samples = 100, n_features = 4, n_optimal = 2, bias = 0.0, noise = 0.5, mean_slope = 1.0):
    """
    This function generates an X and y for regression tasks.
    
    Parameters
    -------
    n_samples : int
        the number of samples to generate
    n_features : int
        the number of columns/features
    n_optimal : int
        the number of useful columns/features, must be <= n_features
    bias : double
        the bias/intercept
    noise : double
        variance
    mean_slope : double
        the approximate slope of optimal values
    
    Returns
    -------
    X : matrix (n_samples, n_features)
        input matrix
    y : vector (n_samples)
        output vector
    """
    # create random X matrix
    X = np.random.rand(n_samples,n_features)
    # good features
    w_opt = mean_slope + np.random.rand(n_optimal)*0.1
    # append bad features
    w = np.hstack((w_opt, np.random.rand(n_features - n_optimal)*0.1))
    # add some noise from normal distribution
    error = np.random.normal(bias, noise, n_samples)
    # apply y = Xw+b+e
    y = np.dot(X,w) + bias + error
    return X,y

In [12]:
# your codes here

### Task 2.

Plot the predicted values $\hat y$ against the actual values $y$ generated as a scatterplot. These can be estimated using $\mathbf{\hat y} \simeq \mathbf{Xw}$.

In [139]:
# your codes here

### Task 3.

Ordinary Least squares is known to suffer from strongly skewed *outliers*. To mitigate this we can apply a regularizing term to the objective minimization function in the form of the $l_2$-norm: $\lvert \lvert \mathbf{w} \rvert \vert_2 \ $:

$$
\mathbf{e} = \min \ \lvert \lvert \mathbf{Xw-y} \rvert \rvert^2 + \lambda \lvert \lvert \mathbf{w} \rvert \vert_2
$$

where $\lambda$ is a hyperparameter to tune the amount of regularization. When derived the optimal minimization of $\bf w$ is: 

$$
\mathbf{w}=(\mathbf{X^TX}+\lambda I)^{-1}\mathbf{X^Ty}
$$

where $I$ refers to the identity matrix.

Write a function *ridge()*, with solves the equation *directly* and which has the same parameters as *least_squares()* with an additional parameter $\lambda=1$ default. Plot $\hat y$ against $y$.

In [138]:
# your codes here

### Task 4.

Calculate the pearson correlation between $\hat y$ and $y$. This is calculated as:

$$
P(x,y)=\frac{\sum_{i=1}^{n} (x_i-\bar x)(y_i-\bar y)}{\sqrt{\sum_{i=1}^n (x_i-\bar x)^2} \sqrt{\sum_{i=1}^n(y_i-\bar y)^2}}
$$

where $\bar x$ and $\bar y$ refer to the mean of each respective vector.

In [137]:
# your codes here

### Task 5.

There are cases of where computing the matrix-inverse is intractable, meaning we cannot solve the matrix *directly*. In this case, we can instead take steps in the direction of the *global minimum* through **gradient descent**. The algorithm works as follows:
1. Initialise $\bf w$ at uniform random, $i = 0$
1. While i < maximum iterations:
    1. Calculate $\Delta_w \mathbf{e}$
    2. Update $w^{(k+1)}=w^{(k)} - \gamma \Delta_w \mathbf{e}$
1. Until convergence

where $\gamma$ is the learning rate.

Write a function *gradient_descent()* using the derivative from least-squares, given $X$, $y$, $\gamma=10^{-3}$ and a number of iterations $K_{max}=10^3$. Save each step and plot $k$ against each weight $w$ (or the mean) to see the minimization in weights.

In [136]:
# your codes here